<a href="https://colab.research.google.com/github/Ingrid-Garcia/Fase-2-usoNLP/blob/main/Pruebas_Unitarias_Reto_Fase_2_Parte_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pruebas Unitarias - Fase 2
Equipo 5:<br>
Ingrid García Hernández, A01754475<br>
Abigail Donají Chavez Rubio, A01747423<br>
Noh Ah Kim Kwon, A01747512<br>
Eduardo Alfredo Ramírez Muñoz, A01754917


# **Definición de librerías**

In [ ]:
pip install pandas openpyxl

In [ ]:
pip install stanza

In [ ]:
!pip install gensim
!pip install --upgrade numpy==1.25.2

In [ ]:
import re

import nltk
from nltk.corpus import stopwords
from nltk.corpus import brown
from nltk.stem import SnowballStemmer
from nltk.util import ngrams
nltk.download('stopwords')

import unicodedata

from sklearn.metrics import pairwise
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import svm

from six import StringIO

from IPython.display import Image


import pydotplus


import stanza
nlp = stanza.Pipeline('es')

import string

from gensim.models import Word2Vec


from matplotlib import pyplot

import unittest

import pandas as pd

import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor    | Package           |
------------------------------------
| tokenize     | combined          |
| mwt          | combined          |
| pos          | combined_charlm   |
| lemma        | combined_nocharlm |
| constituency | combined_charlm   |
| depparse     | combined_charlm   |
| sentiment    | tass2020_charlm   |
| ner          | conll02           |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


# **Prueba limpieza**

In [ ]:
def limpiar_texto(texto):
    '''
    La función de limpiar_texto(texto), tiene como objetivo eliminar el ruido
    dentro de cada elemento de la lista.
    El dato de entrada es una lista de strings y el dato de salida es
    una lista de strings.
    Por cada palabra se debe de normalizar, es decir, se hace caso omiso de los
    acentos y después se revisa cada palabra con respecto a  las reglas
    definidas. Esto se hace por medio de expresiones regulares. Igualmente, se
    eliminan las palabras 'innecesarias',dado que no proporcionan mayor
    información o contexto.
    '''
    #texto.lower()
    tweet = []
    for palabra in texto.split():
      nuevo = palabra
      #Acentos
      nuevo = unicodedata.normalize ('NFKD', nuevo).encode ('ascii', 'ignore').decode ('utf-8', 'ignore')
      for i, j in reglas.items():
        if i == 'espacios_extra':
          nuevo = re.sub(j, ' ', nuevo)
        else:
          nuevo = re.sub(j, '',nuevo)
      if nuevo not in stopwords.words('spanish'):
        tweet.append(nuevo.lower())

    return tweet

reglas = {
    'hashtags': r'#',
    'menciones': r'@',
    'urls': r'http\S+|www.\S+',
    'emojis': r'[^\x00-\x7F]+',
    'puntuacion': r'[^\w\s]',
    'espacios_extras': r'\s+' #Tabs, dobles espacios
}

Casos de prueba

In [ ]:
class TestReglasLimpieza(unittest.TestCase):

    def test_hashtags(self):
        texto = "Amo la #comida y mi #cuerpo"
        salida = limpiar_texto(texto)
        esperado = ["amo", "comida", "cuerpo"]
        self.assertEqual(salida, esperado)

    def test_menciones(self):
        texto = "Gracias @eduardo por tu apoyo"
        salida = limpiar_texto(texto)
        esperado = ["gracias", "eduardo" , "apoyo"]
        self.assertEqual(salida, esperado)

    def test_urls(self):
        texto = "Visita http://teApoyamos.com o www.nutricion.com"
        salida = limpiar_texto(texto)
        esperado = ['visita', '', '']
        self.assertEqual(salida, esperado)

    def test_emojis(self):
        texto = "Estoy muy feliz 😄 por este logro 👏"
        salida = limpiar_texto(texto)
        esperado = ['estoy', 'feliz', '', 'logro', '']
        self.assertEqual(salida, esperado)

    def test_puntuacion(self):
        texto = "¡Hola! ¿Cómo estás? Bien, gracias."
        salida = limpiar_texto(texto)
        esperado = ["hola", "como", "bien", "gracias"]
        self.assertEqual(salida, esperado)

    def test_espacios_extras(self):
        texto = "Hola     mundo, esto    es   una     prueba."
        salida = limpiar_texto(texto)
        esperado = ["hola", "mundo", "prueba"]
        self.assertEqual(salida, esperado)

unittest.main(argv=[''], exit=False, verbosity=2)

test_emojis (__main__.TestReglasLimpieza.test_emojis) ... ok
test_espacios_extras (__main__.TestReglasLimpieza.test_espacios_extras) ... ok
test_hashtags (__main__.TestReglasLimpieza.test_hashtags) ... ok
test_menciones (__main__.TestReglasLimpieza.test_menciones) ... ok
test_puntuacion (__main__.TestReglasLimpieza.test_puntuacion) ... ok
test_urls (__main__.TestReglasLimpieza.test_urls) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.022s

OK


# **Prueba jergas**

In [ ]:
def jerga_proceso(texto):
  '''
    La función de jerga(texto), tiene como objetivo proporcionar información más
    precisa de palabras relaciondas con este desorden alimenticio, mas no lo
    hacen de manera directa.
    El dato de entrada es una lista de lista de strings y el dato de salida es
    una lista de strings.
    Se asuma que la entrada es una lista de palabras y por ende revisa cada una
    de estas para revisar que no se haga caso omiso de ningún caso de jerga.
    '''
  tweet = []
  for palabra in texto:
    if palabra == 'ana':
      palabra = 'anorexia'
    if palabra == 'anorexic':
      palabra = 'anorexia'
    if palabra == 'mía':
      palabra = 'bulimia'
    if palabra == 'bulimic':
      palabra = 'bulimia'
    if palabra == 'mia':
      palabra = 'bulimia'
    if palabra == 'promia':
      palabra = 'bulimia'
    if palabra == 'bulymia':
      palabra = 'bulimia'
    if palabra == 'proana':
      palabra = 'anorexia'
    if palabra == 'styleana':
      palabra = 'anorexia'
    if palabra == 'miaana':
      palabra = 'anorexia'
    if palabra == 'anoreccia':
      palabra = 'anorexia'
    tweet.append(palabra)

  return tweet

Casos de prueba

In [ ]:
class TestJergaProceso(unittest.TestCase):

    def test_palabras_convertidas_a_anorexia(self):
        entrada = ["ana", "anorexic", "proana", "styleana", "miaana", "anoreccia"]
        salida = jerga_proceso(entrada)
        for palabra in salida:
            self.assertEqual(palabra, "anorexia")

    def test_palabras_convertidas_a_bulimia(self):
        entrada = ["mia", "mía", "bulimic", "promia", "bulymia"]
        salida = jerga_proceso(entrada)
        for palabra in salida:
            self.assertEqual(palabra, "bulimia")

    def test_palabras_no_convertidas(self):
        entrada = ["feliz", "comer", "vida"]
        salida = jerga_proceso(entrada)
        self.assertEqual(salida, entrada)

    def test_mixto_convertidos_y_normales(self):
        entrada = ["ana", "feliz", "mia", "vida"]
        salida = jerga_proceso(entrada)
        self.assertEqual(salida, ["anorexia", "feliz", "bulimia", "vida"])

unittest.main(argv=[''], exit=False, verbosity=2)


test_mixto_convertidos_y_normales (__main__.TestJergaProceso.test_mixto_convertidos_y_normales) ... ok
test_palabras_convertidas_a_anorexia (__main__.TestJergaProceso.test_palabras_convertidas_a_anorexia) ... ok
test_palabras_convertidas_a_bulimia (__main__.TestJergaProceso.test_palabras_convertidas_a_bulimia) ... ok
test_palabras_no_convertidas (__main__.TestJergaProceso.test_palabras_no_convertidas) ... ok
test_emojis (__main__.TestReglasLimpieza.test_emojis) ... ok
test_espacios_extras (__main__.TestReglasLimpieza.test_espacios_extras) ... ok
test_hashtags (__main__.TestReglasLimpieza.test_hashtags) ... ok
test_menciones (__main__.TestReglasLimpieza.test_menciones) ... ok
test_puntuacion (__main__.TestReglasLimpieza.test_puntuacion) ... ok
test_urls (__main__.TestReglasLimpieza.test_urls) ... ok

----------------------------------------------------------------------
Ran 10 tests in 0.032s

OK


# **Prueba lematización**

In [ ]:
def lematizacionProc(tweet):
    '''
    La función lematizacionProc(tweet), tiene como objetivo lematizar cada tweet,
    esto con el fin de obtener la forma de cada palabra.
    El dato de entrada es una lista de strings y el dato de salida es
    una lista de strings.
    Dado que es un proceso de lematización se requiere contar con la oración
    completa, para después proceder a unir cada lemma encontrado en una lista.
    '''

    frase = ' '.join(tweet)
    doc = nlp(frase)

    lemma = [word.lemma for sent in doc.sentences for word in sent.words]

    return lemma


Casos de prueba

In [ ]:
class TestLematizacionPasado(unittest.TestCase):

    def test_pasado_regular(self):
        entrada = ["trabajé", "comiendo", "caminamos"]
        esperado = ["trabajar", "comer", "caminar"]
        resultado = lematizacionProc(entrada)
        self.assertEqual(resultado, esperado)

    def test_pasado_irregular(self):
        entrada = ["fui", "tuve", "dije"]
        esperado = ["ser", "tener", "decir"]
        resultado = lematizacionProc(entrada)
        self.assertEqual(resultado, esperado)

    def test_mixto_regulares_irregulares_neutros(self):
        entrada = ["trabajamos", "fue", "dijeron", "casa", "feliz"]
        esperado = ["trabajar", "ser", "decir", "casa", "feliz"]
        resultado = lematizacionProc(entrada)
        self.assertEqual(resultado, esperado)


unittest.main(argv=[''], exit=False, verbosity=2)

test_mixto_convertidos_y_normales (__main__.TestJergaProceso.test_mixto_convertidos_y_normales) ... ok
test_palabras_convertidas_a_anorexia (__main__.TestJergaProceso.test_palabras_convertidas_a_anorexia) ... ok
test_palabras_convertidas_a_bulimia (__main__.TestJergaProceso.test_palabras_convertidas_a_bulimia) ... ok
test_palabras_no_convertidas (__main__.TestJergaProceso.test_palabras_no_convertidas) ... ok
test_mixto_regulares_irregulares_neutros (__main__.TestLematizacionPasado.test_mixto_regulares_irregulares_neutros) ... ok
test_pasado_irregular (__main__.TestLematizacionPasado.test_pasado_irregular) ... ok
test_pasado_regular (__main__.TestLematizacionPasado.test_pasado_regular) ... ok
test_emojis (__main__.TestReglasLimpieza.test_emojis) ... ok
test_espacios_extras (__main__.TestReglasLimpieza.test_espacios_extras) ... ok
test_hashtags (__main__.TestReglasLimpieza.test_hashtags) ... ok
test_menciones (__main__.TestReglasLimpieza.test_menciones) ... ok
test_puntuacion (__main__.T

# **Prueba limpieza total**

In [ ]:
def limpiezaAbsoluta(tweets):
  '''
    La función de limpiezaAbsoluta(tweets), tiene como objetivo hacer el llamdo
    de todas las funciones diferentes para la limpieza de los datos provenientes
    del archivos csv.
    El dato de entrada es una lista de strings y el dato de salida es
    una lista de strings.
    Por cada proceso de limpieza se crea una variable diferente, con el fin de
    evitar confusiones se crean diferentes variables para llevar acabo cada
    llamda de las funciones.
  '''

  limpieza = []
  jerga = []
  lematizacion = []

  #Expresiones regulares
  for tweet in tweets:
    limpieza.append(limpiar_texto(tweet))

  #Jerga
  for tweet in limpieza:
    jerga.append(jerga_proceso(tweet))

  #Lematización

  lematizacion

  for tweet in jerga:
    lematizacion.append(lematizacionProc(tweet))

  return lematizacion

Casos de prueba

In [ ]:
class TestLimpiezaAbsoluta(unittest.TestCase):

    def test_tweet1(self):
        tweets = ["Estoy comiendo con @ana 😊"]
        esperado = ['estar', 'comer', 'anorexia']
        resultado = limpiezaAbsoluta(tweets)[0]
        self.assertEqual(resultado, esperado)

    def test_tweet2(self):
        tweets = ["Dormí feliz con proana y mi cuerpo. 😴"]
        esperado = ['dormi', 'feliz', 'anorexia', 'cuerpo']
        resultado = limpiezaAbsoluta(tweets)[0]
        self.assertEqual(resultado, esperado)

    def test_tweet3(self):
        tweets = ["Amo la #comida y fui a la casa"]
        esperado = ['amar', 'comida', 'casa']
        resultado = limpiezaAbsoluta(tweets)[0]
        self.assertEqual(resultado, esperado)

    def test_tweet4(self):
        tweets = ["Mía dijo que trabajamos bien."]
        esperado = ['bulimia', 'decir', 'trabajar', 'bien']
        resultado = limpiezaAbsoluta(tweets)[0]
        self.assertEqual(resultado, esperado)

unittest.main(argv=[''], exit=False, verbosity=2)


test_mixto_convertidos_y_normales (__main__.TestJergaProceso.test_mixto_convertidos_y_normales) ... ok
test_palabras_convertidas_a_anorexia (__main__.TestJergaProceso.test_palabras_convertidas_a_anorexia) ... ok
test_palabras_convertidas_a_bulimia (__main__.TestJergaProceso.test_palabras_convertidas_a_bulimia) ... ok
test_palabras_no_convertidas (__main__.TestJergaProceso.test_palabras_no_convertidas) ... ok
test_mixto_regulares_irregulares_neutros (__main__.TestLematizacionPasado.test_mixto_regulares_irregulares_neutros) ... ok
test_pasado_irregular (__main__.TestLematizacionPasado.test_pasado_irregular) ... ok
test_pasado_regular (__main__.TestLematizacionPasado.test_pasado_regular) ... ok
test_tweet1 (__main__.TestLimpiezaAbsoluta.test_tweet1) ... ok
test_tweet2 (__main__.TestLimpiezaAbsoluta.test_tweet2) ... ok
test_tweet3 (__main__.TestLimpiezaAbsoluta.test_tweet3) ... ok
test_tweet4 (__main__.TestLimpiezaAbsoluta.test_tweet4) ... ok
test_emojis (__main__.TestReglasLimpieza.test_e

# **Pruebas Bigramas**

In [ ]:
def realizarBigramas(resultadoLimpio):
  '''
  La función de realizarBgramas(resultado_limpio) tiene como objetivo encontar
  por cada tweet sus bigramas correspondintes
  El dato de entrada es una lista de lista de strings y el dato de salida es
  una lista de lista de strings, que contiene los bigramas correspondientes a
  cada tweet.
  '''
  arrBigramas = []
  for elem in resultadoLimpio:
    bigrama = list(ngrams(elem,2))
    arrBigramas.append([" ".join(t) for t in bigrama])

  return arrBigramas

Casos de prueba

In [ ]:
class TestRealizarBigramasAnorexia(unittest.TestCase):

    def test_bigrama_simple(self):
        entrada = [["odio", "comer"]]
        esperado = [["odio comer"]]
        resultado = realizarBigramas(entrada)
        self.assertEqual(resultado, esperado)

    def test_bigrama_multiple(self):
        entrada = [["no", "quiero", "desayunar"]]
        esperado = [["no quiero", "quiero desayunar"]]
        resultado = realizarBigramas(entrada)
        self.assertEqual(resultado, esperado)

    def test_palabra_unica(self):
        entrada = [["delgadez"]]
        esperado = [[]]
        resultado = realizarBigramas(entrada)
        self.assertEqual(resultado, esperado)

    def test_frase_vacia(self):
        entrada = [[]]
        esperado = [[]]
        resultado = realizarBigramas(entrada)
        self.assertEqual(resultado, esperado)

unittest.main(argv=[''], exit=False, verbosity=2)


test_mixto_convertidos_y_normales (__main__.TestJergaProceso.test_mixto_convertidos_y_normales) ... ok
test_palabras_convertidas_a_anorexia (__main__.TestJergaProceso.test_palabras_convertidas_a_anorexia) ... ok
test_palabras_convertidas_a_bulimia (__main__.TestJergaProceso.test_palabras_convertidas_a_bulimia) ... ok
test_palabras_no_convertidas (__main__.TestJergaProceso.test_palabras_no_convertidas) ... ok
test_mixto_regulares_irregulares_neutros (__main__.TestLematizacionPasado.test_mixto_regulares_irregulares_neutros) ... ok
test_pasado_irregular (__main__.TestLematizacionPasado.test_pasado_irregular) ... ok
test_pasado_regular (__main__.TestLematizacionPasado.test_pasado_regular) ... ok
test_tweet1 (__main__.TestLimpiezaAbsoluta.test_tweet1) ... ok
test_tweet2 (__main__.TestLimpiezaAbsoluta.test_tweet2) ... ok
test_tweet3 (__main__.TestLimpiezaAbsoluta.test_tweet3) ... ok
test_tweet4 (__main__.TestLimpiezaAbsoluta.test_tweet4) ... ok
test_bigrama_multiple (__main__.TestRealizarBig

# **Prueba Semejanza Bigramas**

In [ ]:
def semejanzaBigramas(arrBigramas):
  '''
  La función de semejanzasBigramas(arrBigramas) tiene como objetivo definir los
  vectores necesarios para procesar la semejanza entre cada tweet.
  El dato de entrada es una lista de lista de strings y el dato de salida es
  una lista de vectores.
  Se crea una bagOfWords, la cual contiene los bigramas únicos encontrados entre
  todos los tweets, después procede a comparar estos con cada uno de los bigramas
  de cada tweet, agrega un 1 si ha sido igual y 0 de lo contrario.
  '''
  bagOfWords = [];

  for elem in arrBigramas:
    for bigrama in elem:
      if bigrama not in bagOfWords:
        bagOfWords.append(bigrama)

  vectores = []

  for elem in arrBigramas:
    vector = []
    for bigrama in bagOfWords:
      if bigrama in elem:
        vector.append(1) #Lo encontró
      else:
        vector.append(0)
    vectores.append(vector)#No

  return vectores

Casos de prueba

In [ ]:
class TestSemejanzaBigramas(unittest.TestCase):

    def test_bigramas_no_repetidos(self):
        entrada = [
            ["ana odia", "odia comer"],
            ["comer manzana"],
            ["odio mi cuerpo"]
        ]

        esperado = [
            [1, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ]
        resultado = semejanzaBigramas(entrada)
        self.assertEqual(resultado, esperado)

    def test_bigramas_repetidos(self):
        entrada = [
            ["me gusta", "comer sano"],
            ["comer sano", "me gusta"],
            ["comer sano"]
        ]

        esperado = [
            [1, 1],
            [1, 1],
            [0, 1]
        ]
        resultado = semejanzaBigramas(entrada)
        self.assertEqual(resultado, esperado)

    def test_bigramas_unicos(self):
        entrada = [
            ["proana es"],
            ["odio comer"],
            ["vomité todo"]
        ]

        esperado = [
            [1, 0, 0],
            [0, 1, 0],
            [0, 0, 1]
        ]
        resultado = semejanzaBigramas(entrada)
        self.assertEqual(resultado, esperado)

    def test_vacio(self):
        entrada = [[]]
        esperado = [[ ]]  # No hay bigramas, no hay columnas
        resultado = semejanzaBigramas(entrada)
        self.assertEqual(resultado, esperado)

unittest.main(argv=[''], exit=False, verbosity=2)

test_mixto_convertidos_y_normales (__main__.TestJergaProceso.test_mixto_convertidos_y_normales) ... ok
test_palabras_convertidas_a_anorexia (__main__.TestJergaProceso.test_palabras_convertidas_a_anorexia) ... ok
test_palabras_convertidas_a_bulimia (__main__.TestJergaProceso.test_palabras_convertidas_a_bulimia) ... ok
test_palabras_no_convertidas (__main__.TestJergaProceso.test_palabras_no_convertidas) ... ok
test_mixto_regulares_irregulares_neutros (__main__.TestLematizacionPasado.test_mixto_regulares_irregulares_neutros) ... ok
test_pasado_irregular (__main__.TestLematizacionPasado.test_pasado_irregular) ... ok
test_pasado_regular (__main__.TestLematizacionPasado.test_pasado_regular) ... ok
test_tweet1 (__main__.TestLimpiezaAbsoluta.test_tweet1) ... ok
test_tweet2 (__main__.TestLimpiezaAbsoluta.test_tweet2) ... ok
test_tweet3 (__main__.TestLimpiezaAbsoluta.test_tweet3) ... ok
test_tweet4 (__main__.TestLimpiezaAbsoluta.test_tweet4) ... ok
test_bigrama_multiple (__main__.TestRealizarBig

# **Prueba Frecuencia de palabras Clave desde una fuente**

In [ ]:
def palabrasClaveFuente(datosLimpios):
  '''
  La función de palabrasClaveFunte(datosLimpio) tiene como objetivo contabilizar
  las palabras clave relacionadas con el trastorno alimenticio de la anorexia.
  El dato de entrada es una lista de lista de strings y el dato de salida es un
  diccionario con las palabras clave y su conteo.
  La definción de estas palabras proviene de la siguiente fuente:
  http://ilitia.cua.uam.mx:8080/jspui/handle/123456789/884
  '''
  palabrasFrecuentes = {'yo':0, 'nosotros':0, 'social':0, 'amigo':0, 'amigos':0, 'amiga':0, 'humano':0, 'ansiedad':0, 'salud':0, 'sentir':0, 'tristeza':0, 'vida':0, 'sexual':0, 'casa':0, 'hogar':0}

  for tweet in datosLimpios:
    for palabra in tweet:
      if palabra in palabrasFrecuentes:
        palabrasFrecuentes[palabra] += 1;

  return palabrasFrecuentes


Casos de prueba

In [ ]:
class TestPalabrasClaveFuente(unittest.TestCase):

    def test_con_un_tweet(self):
        entrada = [["yo", "siento", "tristeza", "en", "mi", "hogar"]]
        resultado = palabrasClaveFuente(entrada)

        esperado = {
            'yo': 1, 'nosotros': 0, 'social': 0, 'amigo': 0,
            'amigos': 0, 'amiga': 0, 'humano': 0, 'ansiedad': 0,
            'salud': 0, 'sentir': 0, 'tristeza': 1, 'vida': 0,
            'sexual': 0, 'casa': 0, 'hogar': 1
        }

        self.assertEqual(resultado, esperado)

    def test_con_multiples_tweets(self):
        entrada = [
            ["yo", "amigos", "vida", "salud"],
            ["hogar", "sexual", "yo", "sentir"],
            ["tristeza", "ansiedad", "amiga"]
        ]
        resultado = palabrasClaveFuente(entrada)

        esperado = {
            'yo': 2, 'nosotros': 0, 'social': 0, 'amigo': 0,
            'amigos': 1, 'amiga': 1, 'humano': 0, 'ansiedad': 1,
            'salud': 1, 'sentir': 1, 'tristeza': 1, 'vida': 1,
            'sexual': 1, 'casa': 0, 'hogar': 1
        }

        self.assertEqual(resultado, esperado)

    def test_sin_palabras_clave(self):
        entrada = [["pizza", "felicidad", "caminar"]]
        resultado = palabrasClaveFuente(entrada)

        esperado = {
            'yo': 0, 'nosotros': 0, 'social': 0, 'amigo': 0,
            'amigos': 0, 'amiga': 0, 'humano': 0, 'ansiedad': 0,
            'salud': 0, 'sentir': 0, 'tristeza': 0, 'vida': 0,
            'sexual': 0, 'casa': 0, 'hogar': 0
        }

        self.assertEqual(resultado, esperado)

    def test_tweet_vacio(self):
        entrada = [[]]
        resultado = palabrasClaveFuente(entrada)

        esperado = {
            'yo': 0, 'nosotros': 0, 'social': 0, 'amigo': 0,
            'amigos': 0, 'amiga': 0, 'humano': 0, 'ansiedad': 0,
            'salud': 0, 'sentir': 0, 'tristeza': 0, 'vida': 0,
            'sexual': 0, 'casa': 0, 'hogar': 0
        }

        self.assertEqual(resultado, esperado)

unittest.main(argv=[''], exit=False, verbosity=2)

test_mixto_convertidos_y_normales (__main__.TestJergaProceso.test_mixto_convertidos_y_normales) ... ok
test_palabras_convertidas_a_anorexia (__main__.TestJergaProceso.test_palabras_convertidas_a_anorexia) ... ok
test_palabras_convertidas_a_bulimia (__main__.TestJergaProceso.test_palabras_convertidas_a_bulimia) ... ok
test_palabras_no_convertidas (__main__.TestJergaProceso.test_palabras_no_convertidas) ... ok
test_mixto_regulares_irregulares_neutros (__main__.TestLematizacionPasado.test_mixto_regulares_irregulares_neutros) ... ok
test_pasado_irregular (__main__.TestLematizacionPasado.test_pasado_irregular) ... ok
test_pasado_regular (__main__.TestLematizacionPasado.test_pasado_regular) ... ok
test_tweet1 (__main__.TestLimpiezaAbsoluta.test_tweet1) ... ok
test_tweet2 (__main__.TestLimpiezaAbsoluta.test_tweet2) ... ok
test_tweet3 (__main__.TestLimpiezaAbsoluta.test_tweet3) ... ok
test_tweet4 (__main__.TestLimpiezaAbsoluta.test_tweet4) ... ok
test_con_multiples_tweets (__main__.TestPalabra

# **Prueba - Palabras más frecuentes**

In [ ]:
def palabrasClave(datosLimpio):
  '''
  La función de palabrasClave(datosLimpio) tiene como objetivo contabilizar
  las palabras clave relacionadas con el trastorno alimenticio de la anorexia,
  de acuerdo a la información de los tweets ya preprocesaodos.
  El dato de entrada es una lista de lista de strings y el dato de salida es un
  diccionario con las palabras clave y su conteo
  Se revisa cada tweet y las palabras encontradas con el fin de conocer aquellas
  palabras con mayor recurrencia en los tweets.
  '''

  vocabulary = {}
  for tweet in datosLimpio:
    for palabra in tweet:
      if palabra in vocabulary:
        vocabulary[palabra] += 1;
      else:
        vocabulary[palabra] = 1;

  relevantes  = sorted(vocabulary.items(), key=lambda item: item[1], reverse=True)[:10]
  return relevantes



Casos de prueba

In [ ]:
class TestPalabrasClave(unittest.TestCase):

    def test_caso_comun(self):
        entrada = [
            ["ana", "comer", "vida"],
            ["ana", "comer", "feliz"],
            ["comer", "vida", "ana"]
        ]
        esperado = [("ana", 3), ("comer", 3), ("vida", 2), ("feliz", 1)]
        resultado = palabrasClave(entrada)
        self.assertEqual(resultado[:4], esperado)

    def test_empate_en_frecuencia(self):
        entrada = [
            ["ana", "comer"],
            ["vida", "comer"],
            ["ana", "vida"]
        ]
        esperado = [("ana", 2), ("comer", 2), ("vida", 2)]
        resultado = palabrasClave(entrada)
        self.assertEqual(resultado[:3], esperado)

    def test_menos_de_diez_palabras(self):
        entrada = [
            ["una", "dos", "tres"],
            ["cuatro", "cinco"]
        ]
        esperado = [
            ("una", 1), ("dos", 1), ("tres", 1),
            ("cuatro", 1), ("cinco", 1)
        ]
        resultado = palabrasClave(entrada)
        self.assertEqual(resultado, esperado)

    def test_tweet_vacio(self):
        entrada = [[]]
        esperado = []
        resultado = palabrasClave(entrada)
        self.assertEqual(resultado, esperado)

    def test_todo_repetido(self):
        entrada = [
            ["ana", "ana", "ana"],
            ["ana", "ana"]
        ]
        esperado = [("ana", 5)]
        resultado = palabrasClave(entrada)
        self.assertEqual(resultado, esperado)

unittest.main(argv=[''], exit=False, verbosity=2)


test_mixto_convertidos_y_normales (__main__.TestJergaProceso.test_mixto_convertidos_y_normales) ... ok
test_palabras_convertidas_a_anorexia (__main__.TestJergaProceso.test_palabras_convertidas_a_anorexia) ... ok
test_palabras_convertidas_a_bulimia (__main__.TestJergaProceso.test_palabras_convertidas_a_bulimia) ... ok
test_palabras_no_convertidas (__main__.TestJergaProceso.test_palabras_no_convertidas) ... ok
test_mixto_regulares_irregulares_neutros (__main__.TestLematizacionPasado.test_mixto_regulares_irregulares_neutros) ... ok
test_pasado_irregular (__main__.TestLematizacionPasado.test_pasado_irregular) ... ok
test_pasado_regular (__main__.TestLematizacionPasado.test_pasado_regular) ... ok
test_tweet1 (__main__.TestLimpiezaAbsoluta.test_tweet1) ... ok
test_tweet2 (__main__.TestLimpiezaAbsoluta.test_tweet2) ... ok
test_tweet3 (__main__.TestLimpiezaAbsoluta.test_tweet3) ... ok
test_tweet4 (__main__.TestLimpiezaAbsoluta.test_tweet4) ... ok
test_caso_comun (__main__.TestPalabrasClave.tes

# **Prueba TF-IDF**

In [ ]:
def calculoTDIDF(resultadoLimpio):
  '''
   La función de calculoTDIDF(resultadoLimpio) tiene como objetivo identificar
   las palabras clave dentro de todo el conjunto de datos y el peso de estas.
   El dato de entrada es una lista de lista de strings y el dato de salida es un
   dataframe con las palabras clave y su peso correspondiente en cada tweet.
   Se busca en tener una oracion con cada tweet y después se procede a realizar
   el calculo de TF-IDF. Con el fin de tener información más legible se crea
   un dataframe. Y se regresan unicamente las 10 palabras con mayor peso.
  '''
  #Obtener una oracion por cada tweet
  oraciones = [' '.join(palabras) for palabras in resultadoLimpio]

  vectorizer = TfidfVectorizer()

  # Ajustar datos
  matriz = vectorizer.fit_transform(oraciones)

  # Matriz legible
  df = pd.DataFrame(matriz.toarray(), columns=vectorizer.get_feature_names_out())

  tfidf_sums = df.sum(axis=0)

  # Primeras 10 palabras  de TF-IDF
  claves = tfidf_sums.sort_values(ascending=False).head(10)

  return claves



Casos de prueba

In [ ]:
class TestCalculoTFIDF_Simple(unittest.TestCase):

    def test_tres_tweets_distintos(self):
      entrada = [
          ["ana", "odia", "comer"],
          ["comer", "manzana"],
          ["verde", "manzana"]
      ]
      resultado = calculoTDIDF(entrada)
      self.assertTrue(len(resultado) <= 10)
      self.assertIn("comer", resultado.index)
      self.assertIn("manzana", resultado.index)


    def test_repite_palabra_en_un_tweet(self):
        entrada = [
            ["comer", "comer", "comer"],
            ["sano"],
            ["vida"]
        ]
        resultado = calculoTDIDF(entrada)
        palabra_mayor_peso = resultado.index[0]
        self.assertEqual(palabra_mayor_peso, "comer")

    def test_una_palabra_por_tweet(self):
        entrada = [
            ["comer"],
            ["vida"],
            ["ana"]
        ]
        resultado = calculoTDIDF(entrada)
        self.assertEqual(set(resultado.index), {"comer", "vida", "ana"})

unittest.main(argv=[''], exit=False, verbosity=2)


test_repite_palabra_en_un_tweet (__main__.TestCalculoTFIDF_Simple.test_repite_palabra_en_un_tweet) ... ok
test_tres_tweets_distintos (__main__.TestCalculoTFIDF_Simple.test_tres_tweets_distintos) ... ok
test_una_palabra_por_tweet (__main__.TestCalculoTFIDF_Simple.test_una_palabra_por_tweet) ... ok
test_mixto_convertidos_y_normales (__main__.TestJergaProceso.test_mixto_convertidos_y_normales) ... ok
test_palabras_convertidas_a_anorexia (__main__.TestJergaProceso.test_palabras_convertidas_a_anorexia) ... ok
test_palabras_convertidas_a_bulimia (__main__.TestJergaProceso.test_palabras_convertidas_a_bulimia) ... ok
test_palabras_no_convertidas (__main__.TestJergaProceso.test_palabras_no_convertidas) ... ok
test_mixto_regulares_irregulares_neutros (__main__.TestLematizacionPasado.test_mixto_regulares_irregulares_neutros) ... ok
test_pasado_irregular (__main__.TestLematizacionPasado.test_pasado_irregular) ... ok
test_pasado_regular (__main__.TestLematizacionPasado.test_pasado_regular) ... ok
t

# **Prueba Vectorización**

In [ ]:
def vectorizacion(resultadoLimpio):
  '''
  La función de vectorizacion(resultadoLimpio) tiene como objetivo obtener el
  valor vectorial por cada una de las palabaras encontradas en los tweets ya
  preprocesados.
  El dato de entrada es una lista de lista de strings y el dato de salida es un
  objeto de tipo Word2Vec.
  '''

  model = Word2Vec(
    sentences = resultadoLimpio,
    vector_size = 70,
    sg = 1,
    window = 10,
    min_count = 1,
  )
  return model



Casos de prueba

In [ ]:
class TestVectorizacion(unittest.TestCase):

    def test_es_un_modelo_word2vec(self):
        entrada = [["ana", "odia", "comer"], ["comer", "sano"]]
        modelo = vectorizacion(entrada)
        self.assertIsInstance(modelo, Word2Vec)

    def test_modelo_funciona_con_tweet_unico(self):
        entrada = [["comer", "sano", "vida", "feliz", "ana"]]
        modelo = vectorizacion(entrada)
        for palabra in entrada[0]:
            self.assertIn(palabra, modelo.wv.key_to_index)

unittest.main(argv=[''], exit=False, verbosity=2)


test_repite_palabra_en_un_tweet (__main__.TestCalculoTFIDF_Simple.test_repite_palabra_en_un_tweet) ... ok
test_tres_tweets_distintos (__main__.TestCalculoTFIDF_Simple.test_tres_tweets_distintos) ... ok
test_una_palabra_por_tweet (__main__.TestCalculoTFIDF_Simple.test_una_palabra_por_tweet) ... ok
test_mixto_convertidos_y_normales (__main__.TestJergaProceso.test_mixto_convertidos_y_normales) ... ok
test_palabras_convertidas_a_anorexia (__main__.TestJergaProceso.test_palabras_convertidas_a_anorexia) ... ok
test_palabras_convertidas_a_bulimia (__main__.TestJergaProceso.test_palabras_convertidas_a_bulimia) ... ok
test_palabras_no_convertidas (__main__.TestJergaProceso.test_palabras_no_convertidas) ... ok
test_mixto_regulares_irregulares_neutros (__main__.TestLematizacionPasado.test_mixto_regulares_irregulares_neutros) ... ok
test_pasado_irregular (__main__.TestLematizacionPasado.test_pasado_irregular) ... ok
test_pasado_regular (__main__.TestLematizacionPasado.test_pasado_regular) ... ok
t

# **Prueba Transformación**

 Para las siguientes dos pruebas se debe
1. Haber definido previamente la función limpiezaAbsoluta(), que limpia los tweets:
    - Elimina menciones, hashtags, URLs, puntuación, emojis y stopwords.
    - Reemplaza jergas como "ana" o "mía" por "anorexia" o "bulimia".
    - Lematiza las palabras (reduce a su forma base).

2. Haber definido vectorizacion(), que entrena un modelo Word2Vec con los tweets limpios.

In [ ]:
tweets = [
    "Me siento culpable después de comer 😔 @ana_apoyo #culpa",
    "La delgadez es mi meta, no quiero engordar jamás #thinspo",
    "Gracias @ana_power por darme fuerza para resistir la comida #proana",
    "Después de cada atracón, mía me libera... @bulimia_team #purga",
    "Hoy no comeré nada, solo agua y té 🍵 #fasting #control"
]

resultadoLimpio = limpiezaAbsoluta(tweets)
modeloVector = vectorizacion(resultadoLimpio)



In [ ]:
def transformacion(tweets):
  '''
  La función transformacion(tweets) tiene como objetivo que por cada palabra
  encontrada en cada tweet, se tenga un valor vectorial.
  El dato de entrada es una lista de strings y el dato de salida es un vector
  por cada tweet.
  Se busca que por cada tweet tener solo un vector que contenga todos los
  valores de cada palabra.
  '''
  total = 0
  for palabra in tweets:
    total += modeloVector.wv[palabra]
  return total

Casos de prueba

In [ ]:
class TestTransformacionBasica(unittest.TestCase):

    def test_vector_devuelto_es_array(self):
        tweet = resultadoLimpio[0]
        vector = transformacion(tweet)
        self.assertIsInstance(vector, np.ndarray)

    def test_vector_tiene_datos(self):
        tweet = resultadoLimpio[1]
        vector = transformacion(tweet)
        self.assertTrue(any(vector))

    def test_vector_tweet_vacio(self):
        vector = transformacion([])
        self.assertTrue(np.all(vector == 0))

unittest.main(argv=[''], exit=False, verbosity=2)

test_repite_palabra_en_un_tweet (__main__.TestCalculoTFIDF_Simple.test_repite_palabra_en_un_tweet) ... ok
test_tres_tweets_distintos (__main__.TestCalculoTFIDF_Simple.test_tres_tweets_distintos) ... ok
test_una_palabra_por_tweet (__main__.TestCalculoTFIDF_Simple.test_una_palabra_por_tweet) ... ok
test_mixto_convertidos_y_normales (__main__.TestJergaProceso.test_mixto_convertidos_y_normales) ... ok
test_palabras_convertidas_a_anorexia (__main__.TestJergaProceso.test_palabras_convertidas_a_anorexia) ... ok
test_palabras_convertidas_a_bulimia (__main__.TestJergaProceso.test_palabras_convertidas_a_bulimia) ... ok
test_palabras_no_convertidas (__main__.TestJergaProceso.test_palabras_no_convertidas) ... ok
test_mixto_regulares_irregulares_neutros (__main__.TestLematizacionPasado.test_mixto_regulares_irregulares_neutros) ... ok
test_pasado_irregular (__main__.TestLematizacionPasado.test_pasado_irregular) ... ok
test_pasado_regular (__main__.TestLematizacionPasado.test_pasado_regular) ... ok
t

# **Prueba Procesamiento**

In [ ]:
def procesamiento(resultadoLimpio):
  '''
  La función procesamiento(resultadoLimpio) tiene como objetivo descomponer la
  lista tweets.
  El dato de entrada es una lista de lista de strings y el dato de salida es una
  lista de vectores
  Se busca que por cada tweet se obtenga su respectivo valor vectorial, respecto
  a las palabras que contiene.
  '''
  numeroTweets = []
  for tweet in resultadoLimpio:
    numeroTweets.append(transformacion(tweet))
  return numeroTweets


Casos de prueba

In [ ]:
class TestProcesamiento(unittest.TestCase):

    def test_lista_de_vectores(self):
        vectores = procesamiento(resultadoLimpio)
        self.assertEqual(len(vectores), len(resultadoLimpio))

    def test_vector_es_ndarray(self):
        vectores = procesamiento(resultadoLimpio)
        self.assertIsInstance(vectores[0], np.ndarray)

    def test_vector_tiene_datos(self):
        vectores = procesamiento(resultadoLimpio)
        self.assertTrue(any(vectores[0]))

unittest.main(argv=[''], exit=False, verbosity=2)


test_repite_palabra_en_un_tweet (__main__.TestCalculoTFIDF_Simple.test_repite_palabra_en_un_tweet) ... ok
test_tres_tweets_distintos (__main__.TestCalculoTFIDF_Simple.test_tres_tweets_distintos) ... ok
test_una_palabra_por_tweet (__main__.TestCalculoTFIDF_Simple.test_una_palabra_por_tweet) ... ok
test_mixto_convertidos_y_normales (__main__.TestJergaProceso.test_mixto_convertidos_y_normales) ... ok
test_palabras_convertidas_a_anorexia (__main__.TestJergaProceso.test_palabras_convertidas_a_anorexia) ... ok
test_palabras_convertidas_a_bulimia (__main__.TestJergaProceso.test_palabras_convertidas_a_bulimia) ... ok
test_palabras_no_convertidas (__main__.TestJergaProceso.test_palabras_no_convertidas) ... ok
test_mixto_regulares_irregulares_neutros (__main__.TestLematizacionPasado.test_mixto_regulares_irregulares_neutros) ... ok
test_pasado_irregular (__main__.TestLematizacionPasado.test_pasado_irregular) ... ok
test_pasado_regular (__main__.TestLematizacionPasado.test_pasado_regular) ... ok
t